# Modules

In [1]:
import json
# import os
import re
import requests

from bs4 import BeautifulSoup as bs



import configparser
import realstate_utils as ru

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
value1 = config.get('PATH', 'PATH_DATA')
value2 = config.get('URL', 'URL')
value3 = config.get('PAGE', 'page')

print(value1, "\n", value2, "\n", value3)

../Data/Citya/Raw 
 https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc 
 1


# Application